# <center>Big Data &ndash; Moodle Exercises </center>
## <center>Fall 2024 &ndash; Week 9 &ndash; ETH Zurich</center>
## <center>Spark Dataframes and Spark SQL, Moodle exercise</center>

# Preparation for the moodle exercise in Spark

In this jupyter notebook we are going to make the preprocessing part of the dataset that is going to be used in the graded exercise of this week. It will be the same language game dataset as seen in the exercises. If you still do not have created the `confusion-part.json` file, follow these instruction. If you only have it in the `exercise08` folder please copy the `confusion-2014-03-02` folder in the `notebooks` folder of your exam magic box.

1. Move to the `notebooks` folder in the terminal
2. Download the data: <br>
   ```wget https://f003.backblazeb2.com/file/larsyencken-eu-public/greatlanguagegame/confusion-2014-03-02.tbz2``` <br>
   __or__ <br>
   ```curl -O https://f003.backblazeb2.com/file/larsyencken-eu-public/greatlanguagegame/confusion-2014-03-02.tbz2```
3. Extract the data: <br>
   ```tar -jxvf confusion-2014-03-02.tbz2```
4. Change directory to ```confusion-2014-03-02```
5. Extract the part of the dataset that we will work with in this exercise: <br>
   ```head -n 3000000 confusion-2014-03-02.json > confusion-part.json```
## More Info about the data
You can find more information about the dataset (as well as the schema and examples) in the `README.md` file inside the data bundle.

## Instructions:

In every query we ask you for three quantities: the query itself, the result of the query as well as the productivity time. That means the development time of each query (time elapsed from before you start writing the query, until the time at which the correct, final query is ready). Note that the time part of every question is optional and not graded. In order to make easier the time recording we created two functions that do it automatically. Run the cell below in order to import the functions into the current notebook. Then before each query we will have a ```start_exercise()``` cell that you have to run in order to start time recording. After you have finished your query and you are sure about the answer run the ```finish_exercise()``` one to get the time measurement. 

In [1]:
import time

def start_exercise():
    global last
    last = time.time()
    
def finish_exercise():
    global last
    print("This exercise took {0}s".format(int(time.time()-last)))

## <center>1. Spark Dataframes</center>

Write queries for the same dataset as last week, but this time using Spark Dataframes operations (the data loading will take a couple minutes)

### 1.0. Data preprocessing

In [2]:
import json
from pyspark.sql import SparkSession
from pyspark import SparkConf

spark = SparkSession.builder.master('local').getOrCreate()
sc = spark.sparkContext

path = "confusion-2014-03-02/confusion-part.json"
dataset = spark.read.json(path).cache()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/20 16:30:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/20 16:30:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
#test it out
dataset.limit(3).show()

+--------------------+-------+----------+---------+--------------------+---------+
|             choices|country|      date|    guess|              sample|   target|
+--------------------+-------+----------+---------+--------------------+---------+
|[Maori, Mandarin,...|     AU|2013-08-19|Norwegian|48f9c924e0d98c959...|Norwegian|
|[Danish, Dinka, K...|     AU|2013-08-19|    Dinka|af5e8f27cef9e689a...|    Dinka|
|[German, Hungaria...|     AU|2013-08-19|  Turkish|509c36eb58dbce009...|   Samoan|
+--------------------+-------+----------+---------+--------------------+---------+



## Assignment 1
Find the number of games where one of the choices is Italian.

In [4]:
start_exercise()

In [7]:
from pyspark.sql.functions import array_contains
dataset.filter(array_contains("choices", "Italian")).count()

164496

In [11]:
finish_exercise()

This exercise took 306s


## Assignment 2
Return the number of (distinct) countries.

In [18]:
start_exercise()

In [19]:
dataset.select(dataset.country).distinct().count()

188

In [20]:
finish_exercise()

This exercise took 36s


## Assignment 3
Return the sample IDs (i.e., the "sample" field) of the top 2 games played in Switzerland (CH) where the guessed language is correct (equal to the target one) ordered by date (ascending), then by language (descending).

In [24]:
start_exercise()

In [34]:
from pyspark.sql.functions import asc, desc

swiss_games = dataset.filter(dataset.country == 'CH')
swiss_games.filter(swiss_games.guess == swiss_games.target).select("*").orderBy(asc("date"), desc("target")).select("sample").limit(2).show(truncate=False)

+--------------------------------+
|sample                          |
+--------------------------------+
|74b5340a230b1e0c1d45787bc4280b05|
|b7df3f9d67cef259fbcaa5abcad9d774|
+--------------------------------+



In [28]:
finish_exercise()

This exercise took 281s


## Assignment 4
Aggregate all games by guess and target language, ignoring games with correct guesses, count the number of guesses for each group and return the pair with the highest count i.e. the most confused pair.

In [35]:
start_exercise()

In [60]:
from pyspark.sql.functions import count

bad_games = dataset.filter(dataset.target != dataset.guess).select("guess", "target")
bad_games.select("*").groupBy("guess", "target").agg(count("guess").alias("guesses")).orderBy(desc("guesses")).limit(1).show()

+--------+---------+-------+
|   guess|   target|guesses|
+--------+---------+-------+
|Mandarin|Cantonese|    822|
+--------+---------+-------+



In [38]:
finish_exercise()

This exercise took 442s


## Assignment 5
Find the language with the second highest guessed percentage in Switzerland.

In [44]:
start_exercise()

In [45]:
dataset.filter(dataset.country == 'CH').select("guess").groupBy("guess").agg(count("guess").alias("guesses")).orderBy(desc("guesses")).limit(2).show()

+-------+-------+
|  guess|guesses|
+-------+-------+
|Russian|    435|
|Italian|    432|
+-------+-------+



In [46]:
finish_exercise()

This exercise took 198s


## Assignment 6
On which day was any sample played the most ever? Hint: you might need to import `max()`.

In [51]:
start_exercise()

In [58]:
from pyspark.sql.functions import max

dataset.select("date", "sample").groupBy("date").agg(count("sample").alias("samples")).orderBy(desc("samples")).limit(1).show(truncate=False)

+----------+-------+
|date      |samples|
+----------+-------+
|2013-09-04|799435 |
+----------+-------+



In [54]:
finish_exercise()

This exercise took 157s


## <center>2. Spark SQL</center>

Now, for the second weekly quiz, write Spark SQL queries for the same questions as earlier.

### 2.0. Data preprocessing

In [8]:
!pip install sparksql-magic --quiet

In [9]:
%load_ext sparksql_magic

In [10]:
path = "confusion-2014-03-02/confusion-part.json"
dataset = spark.read.json(path).cache()
dataset.createOrReplaceTempView("dataset")

24/11/20 16:37:53 WARN CacheManager: Asked to cache already cached data.        


In [12]:
%%sparksql
-- test it out
SELECT *
FROM dataset
LIMIT 3

choices,country,date,guess,sample,target
"['Maori', 'Mandarin', 'Norwegian', 'Tongan']",AU,2013-08-19,Norwegian,48f9c924e0d98c959d8a6f1862b3ce9a,Norwegian
"['Danish', 'Dinka', 'Khmer', 'Lao']",AU,2013-08-19,Dinka,af5e8f27cef9e689a070b8814dcc02c3,Dinka
"['German', 'Hungarian', 'Samoan', 'Turkish']",AU,2013-08-19,Turkish,509c36eb58dbce009ccf93f375358d53,Samoan


## Assignment 1
Find the number of games where one of the choices is Italian.

In [13]:
start_exercise()

In [16]:
%%sparksql
SELECT COUNT(*)
FROM dataset
WHERE array_contains(dataset.choices, 'Italian')

count(1)
164496


In [17]:
finish_exercise()

This exercise took 158s


## Assignment 2
Return the number of (distinct) countries.

In [21]:
start_exercise()

In [22]:
%%sparksql
SELECT COUNT(DISTINCT(dataset.country))
FROM dataset

count(DISTINCT country)
188


In [23]:
finish_exercise()

This exercise took 42s


## Assignment 3
Return the sample IDs (i.e., the "sample" field) of the top 2 games played in Switzerland (CH) where the guessed language is correct (equal to the target one) ordered by date (ascending), then by language (descending).

In [29]:
start_exercise()

In [32]:
%%sparksql
SELECT dataset.sample
FROM dataset
WHERE dataset.country == 'CH'
AND dataset.target == dataset.guess
ORDER BY dataset.date ASC, dataset.target DESC
LIMIT 2

sample
74b5340a230b1e0c1d45787bc4280b05
b7df3f9d67cef259fbcaa5abcad9d774


In [33]:
finish_exercise()

This exercise took 125s


## Assignment 4
Aggregate all games by guess and target language, ignoring games with correct guesses, count the number of guesses for each group and return the pair with the highest count i.e. the most confused pair.

In [39]:
start_exercise()

In [59]:
%%sparksql
SELECT dataset.guess, dataset.target, COUNT(dataset.guess) AS guesses
FROM dataset
WHERE dataset.target != dataset.guess
GROUP BY dataset.guess, dataset.target
ORDER BY guesses DESC
LIMIT 1

guess,target,guesses
Mandarin,Cantonese,822


In [42]:
finish_exercise()

This exercise took 108s


## Assignment 5
Find the language with the second highest guessed percentage in Switzerland.

In [47]:
start_exercise()

In [49]:
%%sparksql
SELECT dataset.guess, COUNT(dataset.guess) AS guesses
FROM dataset
WHERE dataset.country == 'CH'
GROUP BY dataset.guess
ORDER BY guesses DESC
LIMIT 2

guess,guesses
Russian,435
Italian,432


In [50]:
finish_exercise()

This exercise took 119s


## Assignment 6
On which day was any sample played the most ever?

In [55]:
start_exercise()

In [56]:
%%sparksql
SELECT dataset.date, COUNT(dataset.sample) AS samples
FROM dataset
GROUP BY dataset.date
ORDER BY samples DESC
LIMIT 1

date,samples
2013-09-04,799435


In [57]:
finish_exercise()

This exercise took 71s
